# 1. 개발 환경 소개

- Google Colab CPU 환경에서 진행했습니다.
- 개발 환경은 다음과 같습니다.

In [1]:
import platform
print('- os:',platform.platform())
print('- 운영체제:', end="")
!cat /etc/issue.net
print('- Process information:', platform.processor())
print('- Process Architecture:', platform.machine())
print("- RAM: 12.68GB")

- os: Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
- 운영체제:Ubuntu 18.04.6 LTS
- Process information: x86_64
- Process Architecture: x86_64
- RAM: 12.68GB


# 2. 라이브러리 불러오기 및 경로 지정

### 라이브러리 불러오기

In [2]:
# 전처리 라이브러리
import pandas as pd
import numpy as np
import os

# 분석에 문제가 없는 경고 메시지 숨김
import warnings
warnings.filterwarnings('ignore')

### 라이브러리 버전

In [3]:
print('- ', end="")
!python --version
print('- pandas:', pd.__version__)
print('- numpy:', np.__version__)

- Python 3.7.13
- pandas: 1.3.5
- numpy: 1.21.6


### 경로 설정 및 데이터 불러오기

In [4]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


코드 실행을 위해 데이터가 있는 폴더를 설정합니다.

In [5]:
data_dir = '/content/drive/MyDrive/롯데멤버스_경진대회/3. 안다비젼_ 데이터 및 모델 세이브 파일/'

In [6]:
# 데이터 불러오기
pdde = pd.read_csv(data_dir+'data/LPOINT_BIG_COMP_02_PDDE.csv', parse_dates=['de_dt']) # 상품 구매 정보 - 유통사 상품 구매 내역
pd_clac = pd.read_csv(data_dir+'data/LPOINT_BIG_COMP_04_PD_CLAC.csv') # 상품 분류 정보 - 유통사 상품 카테고리 마스터

# 1. 데이터 분리

- 다음과 같은 형식으로 데이터를 생성합니다.
- user_bundle_train.txt
    - Train 파일
    - 각 행은 '고객ID\t번들ID\n' 입니다.
    - 각 관측값은 고객 u가 번들 b를 적어도 한 번 구매했음을 의미합니다.
- user_item.txt
    - Train 파일
    - 각 행은 '고객ID\t상품ID\n' 입니다.
    - 각 관측값은 고객 u가 상품 b를 적어도 한 번 구매했음을 의미합니다.
- bundle_item.txt
    - Train 파일
    - 각 행은 '번들ID\t상품ID\n' 입니다.
    - 각 관측값은 번들 b 안에 상품 i가 포함됨을 의미합니다.
- Lpoint_data_size.txt
    - 보조 파일
    - 행은 '고객수\t번들수\n아이템수\n' 입니다.
    - 3개의 값은 각각 고객, 번들, 아이템의 수를 의미합니다.
- user_bundle_tune.txt
    - Tune 파일
    - 각 행은 '고객ID\t번들ID\n' 입니다.
    - 각 관측값은 고객 u가 번들 b를 적어도 한 번 구매했음을 의미합니다.
- user_bundle_test.txt
    - Test 파일
    - 각 행은 '고객ID\t번들ID\n' 입니다.
    - 각 관측값은 고객 u가 번들 b를 적어도 한 번 구매했음을 의미합니다.
- 각 상품 대분류명, 상품 소분류명은 번들을 의미합니다.

In [7]:
# 번들(각 상품에 해당하는 대분류명, 소분류명) 생성
data = pdde[['cust','pd_c','de_dt','de_hr']].drop_duplicates(keep='last').reset_index(drop=True)
pd_df = pd_clac[['pd_c', 'clac_hlv_nm']]
pd_df2 = pd_clac[['pd_c', 'clac_mcls_nm']]
df1 = pd.merge(data, pd_df, how='left').rename(columns={'clac_hlv_nm':'bd_nm'})
df2 = pd.merge(data, pd_df2, how='left').rename(columns={'clac_mcls_nm':'bd_nm'})
df = pd.concat([df1, df2]).sort_values(['de_dt','de_hr']).reset_index(drop=True)
df = df[['cust','pd_c','bd_nm']]

# 번들 품목 구매가 2개 이하인 고객 제외
cnt_df = df.groupby(['cust']).count().reset_index()
target = cnt_df['cust'][cnt_df['bd_nm']>2].tolist()
df = df.loc[df['cust'].isin(target)].reset_index(drop=True)

# 고객, 상품, 번들 ID를 숫자 형식으로 변환
df['cust_id'] = df['cust'].astype('category').cat.codes
df['pd_id'] = df['pd_c'].astype('category').cat.codes
df['bd_id'] = df['bd_nm'].astype('category').cat.codes
df = df[['cust_id','bd_id','pd_id']].drop_duplicates().sort_values('cust_id').reset_index(drop=True)

In [8]:
# 데이터 분할 (train:tune:test = 7:1:2)
train = df.groupby('cust_id')['bd_id'].apply(lambda x: x[:int(.7*len(x))])
train = pd.DataFrame(train).reset_index().drop('level_1', axis=1)
tune = df.groupby('cust_id')['bd_id'].apply(lambda x: x[int(.7*len(x)):int(.8*len(x))])
tune = pd.DataFrame(tune).reset_index().drop('level_1', axis=1)
test = df.groupby('cust_id')['bd_id'].apply(lambda x: x[int(.8*len(x)):])
test = pd.DataFrame(test).reset_index().drop('level_1', axis=1)

In [9]:
# 번들-상품 데이터 (bundle_item.txt)
df_bundle_item = df[['bd_id','pd_id']].drop_duplicates().sort_values('bd_id')

# 고객-상품 데이터 (user_item.txt)
df_cust_item = df[['cust_id','pd_id']].drop_duplicates().sort_values('cust_id')

# 각 고객, 번들, 아이템의 수 계산 (data_size.txt)
bd_num = df_bundle_item[['bd_id']].drop_duplicates().count()
pd_num = df_bundle_item[['pd_id']].drop_duplicates().count()
cust_num = df_cust_item[['cust_id']].drop_duplicates().count()
num = pd.DataFrame([[cust_num[0],bd_num[0],pd_num[0]]], dtype=int)

In [10]:
# user_bundle_train.txt
train.to_csv(data_dir+'output/user_bundle_train.txt', header=False, index=False, sep = '\t')

# user_item.txt
df_cust_item.to_csv(data_dir+'output/user_item.txt', header=False, index=False, sep = '\t')

# bundle_item.txt
df_bundle_item.to_csv(data_dir+'output/bundle_item.txt', header=False, index=False, sep = '\t')

# data_size.txt
num.to_csv(data_dir+'output/Lpoint_data_size.txt', header=False, index=False, sep = '\t')

# user_bundle_tune.txt
tune.to_csv(data_dir+'output/user_bundle_tune.txt', header=False, index=False, sep = '\t')

# user_bundle_test.txt
test.to_csv(data_dir+'output/user_bundle_test.txt', header=False, index=False, sep = '\t')